**INSTALL DEPENDENCIES**

In [ ]:
!pip install flask pyngrok transformers accelerate torch --quiet

**CREATE app.py**

In [ ]:
%%writefile app.py
from flask import Flask, render_template, request
from functools import lru_cache
from transformers import pipeline
import re

app = Flask(__name__)

# -----------------------------------------------------------
# 🧠 Load models only once to speed up response times
# -----------------------------------------------------------
@lru_cache(maxsize=1)
def load_t5_model():
    print("🔹 Loading base recipe model (T5)...")
    return pipeline(
        "text2text-generation",
        model="flax-community/t5-recipe-generation"
    )

@lru_cache(maxsize=1)
def load_phi3_model():
    print("🔹 Loading refinement model (Phi-3 Mini)...")
    return pipeline(
        "text-generation",
        model="microsoft/phi-3-mini-4k-instruct",
        dtype="auto",
        device_map="auto"
    )


# -----------------------------------------------------------
# 🔮 Stage 1 — Generate base recipe using T5 model
# -----------------------------------------------------------
def generate_base_recipe(diet, cuisine, goal, time_limit, ingredients, special):
    t5_model = load_t5_model()

    # Prompt instructing T5 to generate raw recipe
    t5_prompt = f"""
You are a recipe generator.

Generate a {diet} {cuisine} recipe using ONLY the following ingredients: {ingredients}.
Do NOT add any new ingredients that are not listed.
You may use small amounts of water, salt, pepper, or oil if absolutely necessary for cooking.
The recipe must be {goal} and cooked in {time_limit}.
{f"Special request: {special}" if special else ""}

Make sure the output includes:
1. A proper recipe title at the top.
2. Clear structure for ingredients and instructions.
"""

    # Generate initial unrefined recipe
    output = t5_model(
        t5_prompt,
        max_new_tokens=250,
        temperature=0.7,
        top_p=0.9
    )[0]["generated_text"].strip()

    return output


# -----------------------------------------------------------
# 🔮 Stage 2 — Refine and format recipe using Phi-3
# -----------------------------------------------------------
def refine_recipe(raw_recipe, ingredients):
    phi_model = load_phi3_model()

    # Structured and strict refinement prompt for Phi-3
    refine_prompt = f"""
<|system|>
You are a professional chef and nutritionist.
You must rewrite and format the given recipe into a clean, clear, and healthy presentation.

❗ Important Rules:
- Use ONLY the ingredients explicitly listed: {ingredients}.
- DO NOT introduce any additional ingredients or seasonings.
- Keep the recipe realistic and tasty using the listed ingredients.
- Generate a creative title if one is missing.

Follow this exact structure:
🍽️ Recipe Title:
🧺 Ingredients:
👨‍🍳 Cooking Instructions:
🥗 Nutritional Info:
💡 Chef’s Tip:

Make sure:
- Quantities are realistic (grams, tsp, tbsp)
- Cooking steps are short, numbered, and clear
- Include approximate nutrition values
- Maintain a friendly, professional tone

<|user|>
{raw_recipe}
<|assistant|>
"""

    # Generate refined recipe
    phi_output = phi_model(
        refine_prompt,
        max_new_tokens=700,
        temperature=0.6,
        top_p=0.9
    )[0]["generated_text"]

    return clean_output(phi_output, refine_prompt)


# -----------------------------------------------------------
# 🧽 Cleanup, fix formatting, handle truncation if required
# -----------------------------------------------------------
def clean_output(text, refine_prompt):

    # Remove special tags and extra system chatter
    text = text.split("<|assistant|>")[-1]
    text = re.sub(r"<\|.*?\|>", "", text)
    text = re.sub(r"You are a professional chef.*", "", text, flags=re.DOTALL)
    text = re.sub(r"\n{2,}", "\n\n", text).strip()

    # Ensure recipe title exists
    if "🍽️ Recipe Title" not in text or re.search(r"🍽️ Recipe Title:\s*$", text):
        text = "🍽️ Recipe Title: Creative Chef’s Choice\n\n" + text

    # Fix incomplete sentence endings
    if text.endswith("for up to 5"):
        text += " days to preserve freshness."
    elif text.endswith("for up to"):
        text += " several days to retain flavor."
    elif not text.endswith(('.', '!', '?')):
        text += "."

    # Retry generation if key sections are missing
    if not any(x in text for x in ["💡 Chef’s Tip", "Nutritional Info", "Cooking Instructions"]):
        phi_model = load_phi3_model()
        cont_prompt = f"{refine_prompt}\nContinue where you left off, completing the recipe neatly."
        cont_output = phi_model(
            cont_prompt,
            max_new_tokens=350,
            temperature=0.6,
            top_p=0.9
        )[0]["generated_text"]
        text += "\n\n" + clean_output(cont_output, refine_prompt)

    return text


# -----------------------------------------------------------
# 🌐 Main route — shows form and recipe result
# -----------------------------------------------------------
@app.route("/", methods=["GET", "POST"])
def home():
    final_recipe = ""

    if request.method == "POST":

        # Collect inputs from HTML form
        diet = request.form["diet"]
        cuisine = request.form["cuisine"]
        goal = request.form["goal"]
        time_limit = request.form["time_limit"]
        ingredients = request.form["ingredients"]
        special = request.form["special"]

        # Stage 1 — Generate base recipe
        raw_recipe = generate_base_recipe(
            diet, cuisine, goal, time_limit, ingredients, special
        )

        # Stage 2 — Refine recipe
        final_recipe = refine_recipe(raw_recipe, ingredients)

    # Render response in HTML page
    return render_template("index.html", recipe=final_recipe)


# -----------------------------------------------------------
# 🚀 Run Flask server
# -----------------------------------------------------------
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)


**CREATE FOLDERS**

In [ ]:
!mkdir -p templates
!mkdir -p static


**templates/index.html**

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <title>🍳 Personalized Recipe Generator</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}" />
</head>

<body>
    <div class="hero-section">
        <div class="overlay"></div>

        <div class="hero-content">
            <h1>🍳 Personalized Recipe Generator</h1>
            <p>Create customized, healthy & delicious recipes using YOUR ingredients.</p>

            <form method="post" class="input-card">

                <input type="text" name="diet" placeholder="Diet type (veg, vegan, non-veg)" required/>

                <input type="text" name="cuisine" placeholder="Cuisine (Indian, Italian, etc.)" required/>

                <input type="text" name="goal" placeholder="Nutrition goal (e.g., high protein)" required/>

                <input type="text" name="time_limit" placeholder="Time limit (e.g., under 20 min)" required/>

                <textarea name="ingredients" rows="3" placeholder="Ingredients (comma-separated)" required></textarea>

                <textarea name="special" rows="2" placeholder="Any special request (optional)"></textarea>

                <button type="submit" class="btn-generate">Generate Recipe 🍽️</button>
            </form>
        </div>
    </div>

    {% if recipe %}
    <div class="result-section fade-in">
        <div class="result-card">
            <h2>🧾 Your Personalized Recipe</h2>
            <p style="white-space: pre-line;">{{ recipe }}</p>
        </div>
    </div>
    {% endif %}
</body>
</html>


**static/style.css**

In [ ]:
%%writefile static/style.css
/* Same theme as your previous project deployment */

@import url('https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&display=swap');

body {
    margin: 0;
    font-family: 'Poppins', sans-serif;
    color: #fff;
}

/* Background banner */
.hero-section {
    height: 100vh;
    position: relative;
    display: flex;
    align-items: center;
    padding-left: 8%;
    background: url('https://res.cloudinary.com/dehfj1nrp/image/upload/v1734949831/kitchen-chef-background.jpg')
        center/cover no-repeat;
}

.overlay {
    position: absolute;
    inset: 0;
    background: rgba(0,0,0,0.55);
}

/* Main card */
.hero-content {
    position: relative;
    z-index: 2;
    width: 500px;
    padding: 40px;
    background: rgba(255,255,255,0.18);
    border-radius: 25px;
    backdrop-filter: blur(7px);
}

h1 {
    font-size: 2.3rem;
    color: #ffdf9e;
    margin-bottom: 10px;
}

/* Form elements */
.input-card {
    display: flex;
    flex-direction: column;
    gap: 15px;
}

input, textarea {
    width: 100%;
    padding: 12px;
    border-radius: 10px;
    border: none;
    background: rgba(255,255,255,0.9);
    color: #111;
}

/* Generate button */
.btn-generate {
    padding: 12px;
    border: none;
    border-radius: 10px;
    background: linear-gradient(135deg, #ff9800, #ff5722);
    color: #fff;
    font-weight: 600;
    cursor: pointer;
    transition: 0.3s;
}

.btn-generate:hover {
    transform: translateY(-3px);
}

/* Result section */
.result-section {
    background: #0d111f;
    padding: 60px;
    display: flex;
    justify-content: center;
}

.result-card {
    max-width: 700px;
    background: rgba(255,255,255,0.12);
    padding: 25px;
    border-radius: 15px;
    color: #e2e5ff;
}

/* Fade animation */
.fade-in {
    animation: fadeInUp 0.7s ease forwards;
}

@keyframes fadeInUp {
    from { opacity: 0; transform: translateY(20px); }
    to { opacity: 1; transform: translateY(0); }
}

**RUN FLASK + NGROK**

In [ ]:
# Kill existing servers
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 617

In [ ]:
# Start Flask in background
!nohup python app.py > flask.log 2>&1 &

In [ ]:
# Start ngrok tunnel
from pyngrok import ngrok, conf

# Enter your NGROK auth token here
conf.get_default().auth_token = "INPUT_YOUR_NGROK_TOKEN_HERE"

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)